In [1]:
# Download Libraries
!pip3 install google-cloud-storage
!pip3 install pyarrow # Apache Arrow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import io
from io import StringIO
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
import os
import datetime
from google.cloud import bigquery
import unicodedata

/Users/karmayangchentenzin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# Initialize a client using the default credentials #Without initializing default credentials first, was unable to create any tables in BigQuery.
client = storage.Client(project="avian-silicon-418821")

# List buckets
buckets = list(client.list_buckets())

# Print bucket names
for bucket in buckets:
    print(bucket.name)  #The correct buckt name from GC is printed.

cis4400_hw1_kyt


In [22]:
# Read Parquet file into DataFrame
parquet_file_path = 'gs://cis4400_hw1_kyt/ny_civil_service_exam.parquet'
df = pd.read_parquet(parquet_file_path)

# Print DataFrame contents
print("DataFrame Contents:")
print(df)
print(df.shape)
otc_df = df.copy()
#Just for testing

DataFrame Contents:
        Exam No  List No   First Name    MI   Last Name  Adj_ FA  \
0          9618   1214.0       GERMAN     A        SOSA    86.67   
1          7001  24935.0       DARREN     L       PAYNE    91.00   
2          7001  11653.0      STEPHEN  None      MANFRE    96.00   
3          2060  15669.0       ELIJAH     T  RICHARDSON    95.71   
4          6601  16886.0          JAY     M        AMES    88.75   
...         ...      ...          ...   ...         ...      ...   
491814      162    640.0        KARIM  None      NUGENT    94.00   
491815      320   3127.0      NICOLAS     C        PUMA    93.61   
491816     2027   4724.0       DILCIA  None       TAPIA    82.44   
491817     2095    291.0  MD ABUBAKAR  None      SIDDIK    70.00   
491818     2545     53.0         LUIS     M       GOMEZ    70.60   

        List Title Code                                 List Title Desc  \
0                 91203                                    BUS OPERATOR   
1            

In [23]:
otc_df.columns

Index(['Exam No', 'List No', 'First Name', 'MI', 'Last Name', 'Adj_ FA',
       'List Title Code', 'List Title Desc', 'Group No', 'List Agency Code',
       'List Agency Desc', 'List Div Code', 'Published Date',
       'Established Date', 'Anniversary Date', 'Extension Date',
       'Veteran Credit', 'Parent Lgy Credit', 'Sibling Lgy Credit',
       'Residency Credit', 'load_date'],
      dtype='object')

In [24]:
otc_df.head()

,Exam No,List No,First Name,MI,Last Name,Adj_ FA,List Title Code,List Title Desc,Group No,List Agency Code,...,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit,load_date
0,9618,1214.0,GERMAN,A,SOSA,86.67,91203,BUS OPERATOR,0,0,...,NaN,None,09/08/2021,09/08/2025,None,None,None,None,None,2024-04-07 02:14:07.932603
1,7001,24935.0,DARREN,L,PAYNE,91.00,70310,FIREFIGHTER,0,0,...,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-04-07 02:14:07.932603
2,7001,11653.0,STEPHEN,None,MANFRE,96.00,70310,FIREFIGHTER,0,0,...,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-04-07 02:14:07.932603
3,2060,15669.0,ELIJAH,T,RICHARDSON,95.71,70112,SANITATION WORKER,0,0,...,NaN,07/12/2023,02/28/2024,02/28/2028,None,None,None,None,Residency Credit,2024-04-07 02:14:07.932603
4,6601,16886.0,JAY,M,AMES,88.75,91207,CONDUCTOR,0,0,...,NaN,None,02/14/2018,02/14/2022,02/14/2025,None,None,None,None,2024-04-07 02:14:07.932603


In [26]:
otc_df['List Div Code'] = otc_df['List Div Code'].fillna('')
otc_df.head()

,Exam No,List No,First Name,MI,Last Name,Adj_ FA,List Title Code,List Title Desc,Group No,List Agency Code,...,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit,load_date
0,9618,1214.0,GERMAN,A,SOSA,86.67,91203,BUS OPERATOR,0,0,...,,None,09/08/2021,09/08/2025,None,None,None,None,None,2024-04-07 02:14:07.932603
1,7001,24935.0,DARREN,L,PAYNE,91.00,70310,FIREFIGHTER,0,0,...,,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-04-07 02:14:07.932603
2,7001,11653.0,STEPHEN,None,MANFRE,96.00,70310,FIREFIGHTER,0,0,...,,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-04-07 02:14:07.932603
3,2060,15669.0,ELIJAH,T,RICHARDSON,95.71,70112,SANITATION WORKER,0,0,...,,07/12/2023,02/28/2024,02/28/2028,None,None,None,None,Residency Credit,2024-04-07 02:14:07.932603
4,6601,16886.0,JAY,M,AMES,88.75,91207,CONDUCTOR,0,0,...,,None,02/14/2018,02/14/2022,02/14/2025,None,None,None,None,2024-04-07 02:14:07.932603


In [4]:

# Specify the path to your service account key file
service_account_key_file = '/Users/karmayangchentenzin/Downloads/Service_Key_HW.json'

# Set up Google Cloud Storage client
storage_client = storage.Client.from_service_account_json(service_account_key_file)

BUCKET_GC = 'cis4400_hw1_kyt'

bucket = storage_client.get_bucket(BUCKET_GC)

# List all blobs in the specified bucket with the given prefix
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)
    blob_data = blob.download_as_string()
    
    try:
        blob_content = blob_data.decode('utf-8')
    except UnicodeDecodeError:
        blob_content = blob_data.decode('latin-1')  # Try decoding with latin-1
    
    df = pd.read_parquet(StringIO(blob_content))
    # Display the shape of the DataFrame
    print(df.shape)


ny_civil_service_exam.parquet


TypeError: binary file expected, got text file

# New Section